# Multiclass Classification of Fatal Car Crashes
Authors:  Ned Kost, Ulises Gomez
Date:  August 25th, 2023

## Goal
Predict the contributing factor for a fatal car crash.  The contributing factor will fall into 3 categories: 
- drunk_driver_involved
- speeding_driver_involved
- other

## TODO
- Load Data into dataframe and clean it
- Identify features to use for the classifier model
- Find a classifier that will do multiclass and probabilty as output.  Preferrably with explainability
- Dockerize the output

In [26]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


## Load and Clean Data

In [53]:
#Load data from Dataframe
df = pd.read_csv('data/fars_train.csv', quoting=1, delimiter=',')
df.head()

,u_id,fatals,a_ct,a_ped_f,a_pedal_f,a_roll,a_hr,a_polpur,month,day,...,owner,deaths,numoccs,impact1,deformed,ve_forms,ve_total,weather,lgt_cond,driver_factor
0,32083,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,10,2,...,Driver (in this crash) Was Registered Owner,1,1.0,Clockpoint 12,Disabling damage,1,1,Clear,Dark - not lighted,other
1,55073,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,6,21,...,Driver (in this crash) Not Registered Owner (o...,1,1.0,Clockpoint 1,Disabling damage,1,1,Clear,Daylight,speeding_driver_involved
2,7458,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,7,14,...,Driver (in this crash) Was Registered Owner,0,1.0,Clockpoint 12,NaN,1,1,Clear,Daylight,other
3,5685,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,9,15,...,Driver (in this crash) Not Registered Owner (o...,0,1.0,Clockpoint 12,Functional damage,1,1,Clear,Dark - not lighted,other
4,9245,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,9,28,...,Driver (in this crash) Was Registered Owner,1,1.0,Clockpoint 9,Disabling damage,1,1,Clear,Dark - not lighted,drunk_driver_involved


In [41]:
def binary_cat(row_value, zero_value):
    if(row_value == zero_value):
        return 0
    else:
        return 1


In [61]:
#Clean up Categorical Data
features = ['fatals', 'a_ped_f', 'day_week', 'a_dow_type','a_tod_type','a_ru','a_inter','a_intsec','a_roadfc','a_junc','a_relrd','age','permvit', 'a_body','owner','deaths','impact1','deformed', 'weather', 'lgt_cond']


cleaned_df = df[features]
#Create column for Pedestrian Fatality from Categorical data
cleaned_df['is_ped_fatality'] = df['a_ped_f'].apply(lambda x: binary_cat(x, 'Other Crash'))
cleaned_df['is_weekend'] = df['a_dow_type'].apply(lambda x: binary_cat(x, 'Weekday'))
cleaned_df['is_night'] = df['a_tod_type'].apply(lambda x: binary_cat(x, 'Daytime'))
cleaned_df['is_urban'] = df['a_ru'].apply(lambda x: binary_cat(x, 'Rural'))
cleaned_df['is_inter'] = df['a_inter'].apply(lambda x: binary_cat(x, 'Non-Interstate'))
cleaned_df['is_intsec'] = df['a_intsec'].apply(lambda x: binary_cat(x, 'Non-Intersection'))


cleaned_df.head()

C:\Users\nedko\AppData\Local\Temp\ipykernel_32452\1705383565.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['is_ped_fatality'] = df['a_ped_f'].apply(lambda x: binary_cat(x, 'Other Crash'))
C:\Users\nedko\AppData\Local\Temp\ipykernel_32452\1705383565.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['is_weekend'] = df['a_dow_type'].apply(lambda x: binary_cat(x, 'Weekday'))
C:\Users\nedko\AppData\Local\Temp\ipykernel_32452\1705383565.py:9: SettingWithCopyWarning: 
A value is tryin

,fatals,a_ped_f,day_week,a_dow_type,a_tod_type,a_ru,a_inter,a_intsec,a_roadfc,a_junc,...,impact1,deformed,weather,lgt_cond,is_ped_fatality,is_weekend,is_night,is_urban,is_inter,is_intsec
0,1,Other Crash,tue,Weekday,Nighttime,Rural,Non-Interstate,Non-Intersection,Minor Collector,Non-Junction,...,Clockpoint 12,Disabling damage,Clear,Dark - not lighted,0,0,1,0,0,0
1,1,Other Crash,wed,Weekday,Daytime,Rural,Non-Interstate,Non-Intersection,Minor Arterial,Non-Junction,...,Clockpoint 1,Disabling damage,Clear,Daylight,0,0,0,0,0,0
2,1,Other Crash,thu,Weekday,Nighttime,Rural,Non-Interstate,Non-Intersection,Major Collector,Non-Junction,...,Clockpoint 12,NaN,Clear,Daylight,0,0,1,0,0,0
3,1,Other Crash,fri,Weekday,Nighttime,Urban,Non-Interstate,Intersection,Principal Arterial – Other,Junction,...,Clockpoint 12,Functional damage,Clear,Dark - not lighted,0,0,1,1,0,1
4,1,Other Crash,fri,Weekday,Nighttime,Rural,Non-Interstate,Non-Intersection,Major Collector,Non-Junction,...,Clockpoint 9,Disabling damage,Clear,Dark - not lighted,0,0,1,0,0,0


# Build Train and Test dataset

In [19]:
X = analyze_df
y = fars_train_df['driver_factor']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=125)

In [25]:
model = MultinomialNB()
model.fit(X,y)

ValueError: could not convert string to float: 'Other Crash'